### Packages

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from os import path

# Usual suspects
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns

# Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Data processing
from brainspace.gradient import GradientMaps
from brainspace.utils.parcellation import reduce_by_labels, map_to_labels

### Custom functions

In [3]:
# Define data input paths
def get_data_dir(scanner, notebooks=False):
    data_dir = '/scratch/tkai/3_projects/1_inprogress/zonaconn' 
    if 'retest' in scanner:
        data_dir = f'{data_dir}/zonaconn-smk_testretest/hcp_retest/results'
    elif 'test' in scanner:
        data_dir = f'{data_dir}/zonaconn-smk_testretest/hcp_test/results'    
    elif scanner == '3T':
        data_dir = f'{data_dir}/zonaconn-smk_3T/results'
    elif scanner == '7T':
        data_dir = f'{data_dir}/zonaconn-smk_7T/results'
    elif notebooks:
        data_dir = f'{data_dir}/notebook'
    
    return data_dir

### Load data

In [4]:
group      = 'tpl-MNI152NLin6Asym'
seed       = 'ZIR'
hemisphere = 'L'
scanner    = '7T' # from '3T', '7T', '3Ttest', '3Tretest'
data_dir   = get_data_dir(scanner)
conn_path  = f'{data_dir}/diffparc/{group}/{group}_hemi-{hemisphere}_label-{seed}_desc-concat_from-group_connMap.npz'

In [5]:
# Subject lists
if 'test' in scanner:
    subjects = pd.read_csv('../config/HCP_TestvRetest_subjects.txt', header=None)
else:
    subjects = pd.read_csv('../config/HCP_3Tv7T_subjects.txt', header=None)

print(f'# of subjects = {len(subjects)} in list')

# of subjects = 169 in list


In [7]:
# Load unsorted data for matrix sizes
data       = np.load(path.join(conn_path))
conn_group = data['conn_group']
mask       = data['mask']
affine     = data['affine']

### Calculate gradients based on group average

In [9]:
n_gradients      = 100
n_gradients_keep = 2

In [10]:
# Concat subjects
conn_group_m = np.moveaxis(conn_group, 0, 2)
conn_concat  = conn_group_m.reshape(
    [conn_group_m.shape[0], conn_group_m.shape[1]*conn_group_m.shape[2]]
)

In [11]:
# Average connectivity matrix
conn_avg = np.nanmean(conn_group, axis=0)
conn_avg_log = np.log(conn_avg)

In [12]:
# Fit gradients based on all subjects
gm = GradientMaps(
    n_components=n_gradients, 
    kernel='normalized_angle',
    approach='dm',
    random_state=0
)

gm.fit(conn_avg_log)

data_dict = {
    'group': {
        'zir': {
            'gradient_value': gm.gradients_,
            'gradient_lambda': gm.lambdas_
        }
    }
}

GradientMaps(kernel='normalized_angle', n_components=100, random_state=0)

#### Save to Nifti volume

In [15]:
G_idx    = np.argwhere(mask)
G_values = {}
for g in range(0,n_gradients_keep):
    G_nii = np.zeros_like(mask)
    G_nii[mask!=0] = gm.gradients_[:,g]
    
    img = nib.Nifti1Image(G_nii, affine=affine)
    nib.save(img, f'output/{scanner}/{seed}_diffparc_g{g+1}_hemi-{hemisphere}_scanner-{scanner}.nii.gz')
    
    G_values[g] = G_nii[G_idx[:,0], G_idx[:,1], G_idx[:,2]].flatten()

### Fit gradients per subject and align to group average

In [20]:
n_subjects = conn_group.shape[0]
# n_subjects = 10

In [21]:
# Fit gradients based on each subject separately
gp = GradientMaps(
    n_components=n_gradients_keep, 
    kernel='normalized_angle',
    approach='dm',
    alignment='procrustes',
    random_state=0
)

In [24]:
gp_subjects = {}
for s in range(n_subjects):
    gp.fit([conn_avg_log, np.log(conn_group[s,:,:])])
    gp_subjects[s] = gp.aligned_[1]
    
    data_dict[f'{subjects.iloc[s,0]}'] = {
        'zir': {
            'gradient_value': gp.aligned_[1],
            'gradient_lambda': gp.lambdas_[1]
        }
    }

### Gradient-weighted cortical maps

In [25]:
# Cortical fsLR atlases
hcp_mmp = {}

for d, density in enumerate(['32k', '59k']):
    h = 'lh' if hemisphere == 'L' else 'rh'
    hcp_mmp[density] = nib.load(
        f'../resources/{h}.hcp-mmp.{density}_fs_LR.label.gii'
    ).darrays[0].data

In [26]:
# Group-level
G_fsLR_lores, G_fsLR_hires = {}, {}
for g in range(n_gradients_keep):
    G_mat            = np.diag(gm.gradients_[:,g])
    gradweighted     = G_mat @ conn_avg_log
    gradweighted_avg = np.mean(gradweighted, axis=0)

    for d in ['32k', '59k']:
        tmp = map_to_labels(
            gradweighted_avg, hcp_mmp[d],
            mask=~np.isin(hcp_mmp[d],0),
            fill=np.nan
        )
        
        if d == '32k':
            G_fsLR_lores[g] = tmp
        else:
            G_fsLR_hires[g] = tmp

        # Write to Gifti surface maps
        gii = nib.gifti.GiftiImage()
        gii.add_gifti_data_array(
            nib.gifti.GiftiDataArray(
                data=tmp.astype(np.float32),
                )
        ) 

        metric_name = f'output/{scanner}/{seed}-CTX_space-fsLR_den-{d}_{hemisphere}_{scanner}_diffparc_g{g+1}-weighted.shape.gii'
        nib.save(gii, metric_name)

data_dict['group']['ctx_lores'] = {
    'vertices': {
        'gradient_value': np.stack(list(G_fsLR_lores.values())).T
    }
}

data_dict['group']['ctx_hires'] = {
    'vertices': {
        'gradient_value': np.stack(list(G_fsLR_hires.values())).T
    }
}

In [27]:
# Subject-level
for s in range(n_subjects):
    G_fsLR_lores, G_fsLR_hires = {}, {}
    
    for g in range(n_gradients_keep):
        G                = data_dict[f'{subjects.iloc[s,0]}']['zir']['gradient_value'][:,g]
        G_mat            = np.diag(G)
        gradweighted     = G_mat @ np.log(conn_group[s,:,:])
        gradweighted_avg = np.mean(gradweighted, axis=0)

        for d in ['32k', '59k']:
            tmp = map_to_labels(
                gradweighted_avg, hcp_mmp[d],
                mask=~np.isin(hcp_mmp[d],0),
                fill=np.nan
            )

            if d == '32k':
                G_fsLR_lores[g] = tmp
            else:
                G_fsLR_hires[g] = tmp

        data_dict[f'{subjects.iloc[s,0]}']['ctx_lores'] = {
            'vertices': {
                'gradient_value': np.stack(list(G_fsLR_lores.values())).T
            }
        }

        data_dict[f'{subjects.iloc[s,0]}']['ctx_hires'] = {
            'vertices': {
                'gradient_value': np.stack(list(G_fsLR_hires.values())).T
            }
        }

### Save data

In [ ]:
# Save data to file
np.save(
    f'output/{scanner}/{seed}_diffparc_hemi-{hemisphere}_scanner-{scanner}_gradients-data.npy',
    data_dict
)